#### 各パッケージのインストール，データ，hyperparametersの読み込み

In [1]:

#必要なパッケージのインポート

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import reservoirpy as rpy

from scipy.integrate import solve_ivp
import pandas as pd
from reservoirpy.observables import nrmse, rsquare

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from reservoirpy.datasets import to_forecasting

rpy.verbosity(0)

import os

from reservoirpy.nodes import Reservoir, Ridge
from reservoirpy.datasets import mackey_glass

import joblib


# just a little tweak to center the plots, nothing to worry about
from IPython.core.display import HTML
HTML("""
<style>
.img-center {
    display: block;
    margin-left: auto;
    margin-right: auto;
    }
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
    }
</style>
""")

rpy.set_seed(42)

%time


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


### 7. Generative Modelのうち，外力のデータのみ実データで更新し続ける．

期待としては，X, Yの精度も上がるということである．

In [2]:
def plot_generation(X_gen, X_t, nb_generations, warming_out=None, warming_inputs=None, seed_timesteps=0):
    plt.figure(figsize=(15, 5))
    if warming_out is not None:
        plt.plot(np.vstack([warming_out, X_gen]), label="Generated timeseries")
    else:
        plt.plot(X_gen, label="Generated timeseries")

    plt.plot(np.arange(nb_generations) + seed_timesteps, X_t, linestyle="--", label="Real timeseries")

    # `warming_inputs`のサイズを検証して調整します。
    if warming_inputs is not None and len(warming_inputs) > seed_timesteps:
        # `seed_timesteps`に合わせてサイズを調整
        warming_inputs = warming_inputs[:seed_timesteps]
        plt.plot(warming_inputs, linestyle="--", label="Warmup")

    plt.plot(np.arange(nb_generations) + seed_timesteps, np.abs(X_t - X_gen), label="Absolute deviation")

    if seed_timesteps > 0:
        plt.fill_between([0, seed_timesteps], *plt.ylim(), facecolor='lightgray', alpha=0.5, label="Warmup period")

    plt.legend()
    plt.show()


In [3]:
N = 10000
iss = 0.08827385538440446
lr = 0.33989147591891816
ridge =  1.30084237042553e-08
seed = 3
sr = 0.568437354122632
forecast = 1

In [4]:
def reset_esn():
    from reservoirpy.nodes import Reservoir, Ridge

    reservoir = Reservoir(N, 
                      sr=sr, 
                      lr=lr, 
                      input_scaling=iss, 
                      seed=seed)
    readout = Ridge(ridge=ridge)

    return reservoir >> readout


In [5]:
# 時系列データの比較を行う関数
def compare_timeseries_multidimensional(X_gen, X_t):
    num_dimensions = X_gen.shape[1]  # 次元数を取得
    for dim in range(num_dimensions):
        plt.figure(figsize=(15, 5))
        plt.plot(X_gen[:, dim][:1000], label="Generated timeseries", linewidth=2)
        plt.plot(X_t[:, dim][:1000], label="Real timeseries", linewidth=2)
        plt.plot(np.abs(X_t[:, dim][:1000] - X_gen[:, dim][:1000]), label="Absolute deviation", linestyle='--')
        plt.plot(nrmse(X_gen[:, dim][:1000], X_t[:, dim][:1000]), label=f"nrmse:{nrmse(X_gen[:, dim][:1000], X_t[:, dim][:1000])}", linewidth=2)
        plt.plot(rsquare(X_gen[:, dim][:1000], X_t[:, dim][:1000]), label=f"rsquare:{rsquare(X_gen[:, dim][:1000], X_t[:, dim][:1000])}", linewidth=2)

        plt.title(f'Comparison of Generated and Real Timeseries (Dimension {dim+1})')
        plt.xlabel('Time Steps')
        plt.ylabel('Values')
        plt.legend()        
        plt.show()

3~4周期先までの予測．

### 外力の形が異なるものに対して予測が可能なのかを調べよう

In [6]:
dir_name_1 = f"SE_result"
os.makedirs(dir_name_1, exist_ok=True)

dir_name_2 = f"SE_val"
os.makedirs(dir_name_2, exist_ok=True)

In [7]:
for i in range(-12, 13):
# CSVファイルにデータを保存
    filename_with_force = f"data_def/VDP_{i}.csv"

    # CSVファイルを読み込む
    data_loaded_with_force = pd.read_csv(filename_with_force)

    # CSVから値を抽出してNumpy配列に格納
    X = data_loaded_with_force[['x', 'y', 'P_shifted']].values

    X.shape

    n,m = X.shape

    N = 10000
    iss = 0.5768591838727728
    lr = 0.4304317721067772
    ridge = 0.00041298826582042474
    seed = 3
    sr = 0.866615511675912
    forecast = 1

    train_len = 40000
    start_time = 0
    test_length = 20000
    nb_generations = 10000

    seed_timesteps = test_length 

    nb_generations = len(X) - (train_len + test_length)

    seed_timesteps = test_length 

    esn = reset_esn()

    X_train = X[start_time:start_time+train_len]
    y_train = X[start_time+1 :start_time+train_len + 1]

    X_test = X[start_time+train_len : start_time+train_len + seed_timesteps]
    y_test = X[start_time+train_len + 1: start_time+train_len + seed_timesteps + 1]

    X_evolve = X[start_time+train_len + seed_timesteps:]

    esn = esn.fit(X_train, y_train)

    warming_inputs = X_test

    warming_out = esn.run(warming_inputs, reset=True)  # warmup
    #warming_outはX_test[seed_timesteps]を近似する．

    X_gen = np.zeros((nb_generations, m))
    y = warming_out[-1] 
    y = y.reshape(1, -1) 

    for t in range(nb_generations):  
        y[:, 2:3] = X_evolve[t, 2:3] #外力にあたる[:, 2:3]に実測値を代入する．
        y = esn(y) #ESNで1回=0.1ステップ先を予測する．
        X_gen[t, :] = y #配列に記録していく
                
    X_t = X_evolve[: nb_generations]

    import numpy as np

    # X_genを適当なファイル名で保存する場合
    file_name_1 = f"{dir_name_1}/CmaEs_shift{i}.csv"

    # X_tをCSVファイルに書き出す
    np.savetxt(file_name_1, X_t, delimiter=',')

    # X_tを適当なファイル名で保存する場合
    file_name_2 = f"{dir_name_2}/CmaEs_shift{i}.csv"

    # X_tをCSVファイルに書き出す
    np.savetxt(file_name_2, X_gen, delimiter=',')